In [1]:
import nltk
import time
import numpy as np
import pandas as pd
from nltk import pos_tag
from sklearn.svm import SVC
from string import punctuation 
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import NaiveBayesClassifier as NBC
from nltk.stem import WordNetLemmatizer as WNL
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
time1 = time.time()
data = pd.read_csv("train.csv")
data.head()

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)


In [3]:
doc = data["text"]

In [4]:
all_doc = []
for i in range(len(data)):
    all_doc.append((word_tokenize(data["text"][i]), data["airline_sentiment"][i]))
all_doc[:10]

[(['@',
   'SouthwestAir',
   'I',
   'am',
   'scheduled',
   'for',
   'the',
   'morning',
   ',',
   '2',
   'days',
   'after',
   'the',
   'fact',
   ',',
   'yes',
   '..',
   'not',
   'sure',
   'why',
   'my',
   'evening',
   'flight',
   'was',
   'the',
   'only',
   'one',
   'Cancelled',
   'Flightled'],
  'negative'),
 (['@',
   'SouthwestAir',
   'seeing',
   'your',
   'workers',
   'time',
   'in',
   'and',
   'time',
   'out',
   'going',
   'above',
   'and',
   'beyond',
   'is',
   'why',
   'I',
   'love',
   'flying',
   'with',
   'you',
   'guys',
   '.',
   'Thank',
   'you',
   '!'],
  'positive'),
 (['@',
   'united',
   'Flew',
   'ORD',
   'to',
   'Miami',
   'and',
   'back',
   'and',
   'had',
   'great',
   'crew',
   ',',
   'service',
   'on',
   'both',
   'legs',
   '.',
   'THANKS'],
  'positive'),
 (['@',
   'SouthwestAir',
   '@',
   'dultch97',
   'that',
   "'s",
   'horse',
   'radish',
   '😤🐴'],
  'negative'),
 (['@',
   'united',
   's

In [5]:
stop_words = stopwords.words("english")
punc = list(punctuation)
stop_words+=punc
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [6]:
lemm = WNL()

In [7]:
def get_simple_pos(tag): #since pos tag are comlicated make it simpler
    if tag.startswith("J"):
        return wordnet.ADJ
    elif tag.startswith("V"):
        return wordnet.VERB
    elif tag.startswith("N"):
        return wordnet.NOUN
    elif tag.startswith("R"):
        return wordnet.ADV
    else:
        wordnet.NOUN

In [8]:
def cleaning(text):
    output_words = []
    for word in text:
        if word.lower() not in stop_words and len(word)>2:
            ps = pos_tag(word)
            wn = str(get_simple_pos(ps[0][1]))
            clean_word = lemm.lemmatize(word, pos = wordnet.NOUN)
            output_words.append(clean_word)
    return output_words

In [9]:
final_doc = [(cleaning(text), cat) for text, cat in all_doc]
final_doc[:2]

[(['SouthwestAir',
   'scheduled',
   'morning',
   'day',
   'fact',
   'yes',
   'sure',
   'evening',
   'flight',
   'one',
   'Cancelled',
   'Flightled'],
  'negative'),
 (['SouthwestAir',
   'seeing',
   'worker',
   'time',
   'time',
   'going',
   'beyond',
   'love',
   'flying',
   'guy',
   'Thank'],
  'positive')]

In [10]:
all_word = []
for i in final_doc:
    all_word +=i[0]
len(all_word)

110664

In [11]:
freq = nltk.FreqDist(all_word)
freq.most_common(15)

[('united', 2928),
 ('flight', 2806),
 ('AmericanAir', 2208),
 ('USAirways', 2173),
 ('SouthwestAir', 1801),
 ('JetBlue', 1693),
 ("n't", 1529),
 ('get', 988),
 ('http', 906),
 ('hour', 853),
 ('Cancelled', 800),
 ('service', 708),
 ('time', 678),
 ('customer', 663),
 ('...', 647)]

In [12]:
features = [i[0] for i in freq.most_common(1000)]
features

['united',
 'flight',
 'AmericanAir',
 'USAirways',
 'SouthwestAir',
 'JetBlue',
 "n't",
 'get',
 'http',
 'hour',
 'Cancelled',
 'service',
 'time',
 'customer',
 '...',
 'help',
 'bag',
 'Flight',
 'plane',
 'amp',
 'hold',
 'would',
 'call',
 'Thanks',
 'need',
 'thanks',
 'one',
 'day',
 'back',
 'gate',
 'Flightled',
 'airline',
 'still',
 'VirginAmerica',
 'delayed',
 "'ve",
 'got',
 'delay',
 'seat',
 'like',
 'Late',
 'phone',
 'agent',
 'guy',
 'today',
 'ticket',
 'know',
 'fly',
 'minute',
 'waiting',
 'change',
 'please',
 'make',
 'United',
 'airport',
 'Thank',
 'way',
 'could',
 'min',
 'wait',
 'trying',
 "'re",
 'home',
 'take',
 'tomorrow',
 'going',
 'flying',
 'thank',
 'weather',
 'last',
 'people',
 'even',
 'check',
 'great',
 'work',
 'told',
 "'ll",
 'want',
 'getting',
 'issue',
 'say',
 'never',
 'email',
 'due',
 'another',
 'luggage',
 'lost',
 'really',
 'number',
 'good',
 'Flighted',
 'see',
 'trip',
 'travel',
 'hr',
 'much',
 'someone',
 'first',
 'bag

In [13]:
def get_dict(text):
    curr_feature = {}
    word_set = (text)
    for w in features:
        curr_feature[w] = w in word_set
    return curr_feature

In [14]:
training_data = [(get_dict(words), cat) for words, cat in final_doc]
training_data[:5]

[({'united': False,
   'flight': True,
   'AmericanAir': False,
   'USAirways': False,
   'SouthwestAir': True,
   'JetBlue': False,
   "n't": False,
   'get': False,
   'http': False,
   'hour': False,
   'Cancelled': True,
   'service': False,
   'time': False,
   'customer': False,
   '...': False,
   'help': False,
   'bag': False,
   'Flight': False,
   'plane': False,
   'amp': False,
   'hold': False,
   'would': False,
   'call': False,
   'Thanks': False,
   'need': False,
   'thanks': False,
   'one': True,
   'day': True,
   'back': False,
   'gate': False,
   'Flightled': True,
   'airline': False,
   'still': False,
   'VirginAmerica': False,
   'delayed': False,
   "'ve": False,
   'got': False,
   'delay': False,
   'seat': False,
   'like': False,
   'Late': False,
   'phone': False,
   'agent': False,
   'guy': False,
   'today': False,
   'ticket': False,
   'know': False,
   'fly': False,
   'minute': False,
   'waiting': False,
   'change': False,
   'please': False

In [15]:
data_test = pd.read_csv("test.csv")
test_doc = data_test["text"]

all_doc_test = []
for i in range(len(test_doc)):
    all_doc.append(word_tokenize(data_test["text"][i]))

testing_data = [get_dict(words) for words in all_doc]
testing_data[:5]

[{'united': False,
  'flight': False,
  'AmericanAir': False,
  'USAirways': False,
  'SouthwestAir': False,
  'JetBlue': False,
  "n't": False,
  'get': False,
  'http': False,
  'hour': False,
  'Cancelled': False,
  'service': False,
  'time': False,
  'customer': False,
  '...': False,
  'help': False,
  'bag': False,
  'Flight': False,
  'plane': False,
  'amp': False,
  'hold': False,
  'would': False,
  'call': False,
  'Thanks': False,
  'need': False,
  'thanks': False,
  'one': False,
  'day': False,
  'back': False,
  'gate': False,
  'Flightled': False,
  'airline': False,
  'still': False,
  'VirginAmerica': False,
  'delayed': False,
  "'ve": False,
  'got': False,
  'delay': False,
  'seat': False,
  'like': False,
  'Late': False,
  'phone': False,
  'agent': False,
  'guy': False,
  'today': False,
  'ticket': False,
  'know': False,
  'fly': False,
  'minute': False,
  'waiting': False,
  'change': False,
  'please': False,
  'make': False,
  'United': False,
  'airpo

In [16]:
model = NBC.train(training_data)
model.show_most_informative_features(15) 

Most Informative Features
                   Great = True           positi : neutra =     35.7 : 1.0
                favorite = True           positi : negati =     34.2 : 1.0
                    hold = True           negati : positi =     32.9 : 1.0
                    Wall = True           neutra : negati =     26.5 : 1.0
                 helpful = True           positi : neutra =     26.3 : 1.0
                 awesome = True           positi : negati =     25.3 : 1.0
               excellent = True           positi : negati =     23.6 : 1.0
                   thank = True           positi : negati =     23.0 : 1.0
                   Thank = True           positi : negati =     21.8 : 1.0
                    Love = True           positi : neutra =     21.1 : 1.0
                  online = True           negati : positi =     19.6 : 1.0
               wonderful = True           positi : negati =     19.0 : 1.0
                      hr = True           negati : positi =     16.7 : 1.0

In [17]:
svc = SVC()
model = SklearnClassifier(svc) 

In [18]:
model.train(training_data)
nltk.classify.accuracy(model, training_data)

0.8867941712204007

In [19]:
rfc = RFC()
model1 =  SklearnClassifier(rfc)
model1.train(training_data)
nltk.classify.accuracy(model1, training_data)

0.9844262295081967

In [20]:
time2 = time.time()
print(time2- time1)

1635.7866928577423
